#### Sprint 11 Project

In [43]:
import pandas as pd

from sklearn.neighbors import NearestNeighbors

from sklearn.model_selection import train_test_split, GridSearchCV

from sklearn.linear_model import LogisticRegression

import numpy as np

from sklearn import metrics

from sklearn.dummy import DummyClassifier

In [2]:
try:
    insurance_df = pd.read_csv('/datasets/insurance_us.csv')

except:
    insurance_df = pd.read_csv('insurance_us.csv')

#### EDA/ Data Preprocessing

In [3]:
insurance_df.head()

,Gender,Age,Salary,Family members,Insurance benefits
0,1,41,49600,1,0
1,0,46,38000,1,1
2,0,29,21000,0,0
3,0,21,41700,2,0
4,1,28,26100,0,0


In [4]:
insurance_df = insurance_df.rename(columns={"Gender": "gender", "Age": "age", "Salary": "salary", "Family members": "family_members", "Insurance benefits": "insurance_benefits"})

In [5]:
insurance_df.isna().sum()

gender                0
age                   0
salary                0
family_members        0
insurance_benefits    0
dtype: int64

In [6]:
insurance_df.duplicated().sum()

153

In [7]:
insurance_df.drop_duplicates(inplace=True)

insurance_df.duplicated().sum()

0

In [8]:
insurance_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4847 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype
---  ------              --------------  -----
 0   gender              4847 non-null   int64
 1   age                 4847 non-null   int64
 2   salary              4847 non-null   int64
 3   family_members      4847 non-null   int64
 4   insurance_benefits  4847 non-null   int64
dtypes: int64(5)
memory usage: 227.2 KB


In [18]:
#Use np.random.RandomState() in order to create a random state for the models.
state = np.random.RandomState(12345)

#### Task 1:
Find customers who are similar to a given customer. This will help the company's agents with marketing.

documentation from Daniel:
https://scikit-learn.org/0.15/modules/generated/sklearn.neighbors.NearestNeighbors.html

scroll down and see examples of nearest neighbors algorithm in practice

no need for k neighbors classifier need nearest neighbors method in sklearn.
no need for target variable
given a row of data after fitting entire data, check what are n most similar rows.
can fit df into a nearest neighbors a 

In [9]:
t1_model= NearestNeighbors(n_neighbors=2)

In [10]:
t1_model.fit(insurance_df)

NearestNeighbors(n_neighbors=2)

In [11]:
t1_model.radius_neighbors(insurance_df)

(array([array([0., 1.]), array([0.]), array([0.]), ..., array([0.]),
        array([0.]), array([0., 1.])], dtype=object),
 array([array([   0, 1992], dtype=int64), array([1], dtype=int64),
        array([2], dtype=int64), ..., array([4844], dtype=int64),
        array([4845], dtype=int64), array([4846, 3420], dtype=int64)],
       dtype=object))

#### Task 2
Predict whether a new customer is likely to receive an insurance benefit. Can a trained prediction model do better than an untrained dummy model? Can it do worse? Explain your logic.

In [12]:
insurance_df['recieved_benefits'] = (insurance_df['insurance_benefits'] > 0).astype(int)

In [30]:
t2_features = insurance_df.drop(['recieved_benefits'], axis=1)
t2_target = insurance_df['recieved_benefits']

t2_x_train, t2_x_valid, t2_y_train, t2_y_valid = train_test_split(t2_features, t2_target, test_size=0.3, random_state=12345)

In [45]:
t2_dummy_model = DummyClassifier(random_state=state)

t2_dummy_model.fit(t2_x_train, t2_y_train)

DummyClassifier(random_state=RandomState(MT19937) at 0x1A57B63C740)

In [48]:
t2_dummy_predicted = t2_dummy_model.predict(t2_x_valid)

t2_dummy_cmatrix = metrics.confusion_matrix(t2_y_valid, t2_dummy_predicted)

print(f"Task 2 Dummy Classifier Confusion Matrix: \n{t2_dummy_cmatrix}")

Task 2 Dummy Classifier Confusion Matrix: 
[[1287    0]
 [ 168    0]]


In [19]:
t2_trained_model = LogisticRegression(random_state=state)

In [20]:
#Create a parameter grid for the Logisitic Regression Model.
log_r_param_grid = {
    'penalty': ['l1', 'l2'],
    'C': [0.01, 0.1, 1, 10, 100],
    'solver': ['liblinear', 'saga']
}

In [22]:
log_r_gsearch = GridSearchCV(estimator=t2_trained_model, param_grid=log_r_param_grid, cv=5, scoring='accuracy')

log_r_gsearch.fit(t2_x_train, t2_y_train)

c:\Users\HP\anaconda3\Lib\site-packages\sklearn\linear_model\_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
c:\Users\HP\anaconda3\Lib\site-packages\sklearn\linear_model\_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
c:\Users\HP\anaconda3\Lib\site-packages\sklearn\linear_model\_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
c:\Users\HP\anaconda3\Lib\site-packages\sklearn\linear_model\_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
c:\Users\HP\anaconda3\Lib\site-packages\sklearn\linear_model\_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
c:\Users\HP\anaconda3\Lib\site-packages\sklearn\linear_model\_sag.py:349: ConvergenceWarning: The max_iter was reached which 

GridSearchCV(cv=5,
             estimator=LogisticRegression(random_state=RandomState(MT19937) at 0x1A57B63C740),
             param_grid={'C': [0.01, 0.1, 1, 10, 100], 'penalty': ['l1', 'l2'],
                         'solver': ['liblinear', 'saga']},
             scoring='accuracy')

In [27]:
print(f"Logistic Regression Model Best Score: {log_r_gsearch.best_score_}")
print(f"Logistic Regression Model Best Parameters:\n{log_r_gsearch.best_params_}")

Logistic Regression Model Best Score: 1.0
Logistic Regression Model Best Parameters:
{'C': 0.1, 'penalty': 'l1', 'solver': 'liblinear'}


In [29]:
t2_trained_model = LogisticRegression(C=0.1, penalty="l1", solver='liblinear', random_state=state)

t2_trained_model.fit(t2_x_train, t2_y_train)

LogisticRegression(C=0.1, penalty='l1',
                   random_state=RandomState(MT19937) at 0x1A57B63C740,
                   solver='liblinear')

In [40]:
t2_trained_predict = t2_trained_model.predict(t2_x_valid)

t2_trained_cmatrix = metrics.confusion_matrix(t2_y_valid, t2_trained_predict)

print(f"Logistic Regression Trained Model Confusion Matrix: \n{t2_trained_cmatrix}")

Logistic Regression Trained Model Confusion Matrix: 
[[1287    0]
 [   0  168]]
